# **Weighted ROMs**
Weighted ROMs are a way to insert data information in the reduced setting. They are used to accelarate statistics and Monte Carlo simulations.

In this exercise you can practise with weighted-POD (wPOD) for Stokes problem.

Let us import gedim!



In [ ]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

In [ ]:
!mkdir -p release
%cd release 
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

In [ ]:
import numpy as np
import GeDiM4Py as gedim
from scipy.sparse.linalg import splu
import time

In [ ]:
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## The parametric version of the Stokes problem

Solve the Stokes equation on square ${\Omega} = (0, 1) \times (0, 1)$

$$
\begin{cases}
-\mu_1 \nabla \cdot (\nabla \mathbf{u}) + \nabla p = \mathbf{f}(\mu_2) & \text{in } \Omega\\
(\nabla \cdot \mathbf{u}) = 0 & \text{in } \Omega\\
u = 0 & \text{in } ∂ \Omega
\end{cases}
$$

where $\nu$ is the **viscosity**, $\mathbf{u} = (u_1, u_2)$ is the **speed** and $p$ is the **pressure**, with $\mathbf{f}$ a parametric version of the forcing term of Lab10.

In this case $\boldsymbol \mu$ is a random variable and each parameter has is peculiar probability distribution $\rho(\mu_1)$ and $\rho(\mu_2)$. The two parameters are assumed to be independent and identically distributed random variables.

In terms of high-fidelity simulations, nothing changes.

In [ ]:
def Stokes_V():
	return 1.0

def Stokes_v(numPoints, points):
	values = np.ones(numPoints) * Stokes_V()
	return values.ctypes.data

def Stokes_advection_1(numPoints, points):
	values = np.zeros((2, numPoints), order='F')
	values[0,:] = 1.0
	return values.ctypes.data

def Stokes_advection_2(numPoints, points):
	values = np.zeros((2, numPoints), order='F')
	values[1,:] = 1.0
	return values.ctypes.data
############## Forcing term WRT mu_2 #####################
def Stokes_f_1(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = - ((mu_2**3) * np.pi * np.pi * np.cos((mu_2**2)* np.pi * matPoints[0,:]) - (mu_2**2) * np.pi * np.pi) * np.sin(mu_2 * np.pi * matPoints[1,:]) * np.cos(mu_2 * np.pi * matPoints[1,:]) + (+mu_2 * np.pi * np.cos(mu_2 * np.pi * matPoints[0,:]) * np.cos(mu_2 * np.pi * matPoints[1,:]))
	return values.ctypes.data

def Stokes_f_2(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = - (-(mu_2**3) * np.pi * np.pi * np.cos((mu_2**2) * np.pi * matPoints[1,:]) + (mu_2**2) * np.pi * np.pi) * np.sin(mu_2 * np.pi * matPoints[0,:]) * np.cos(mu_2 * np.pi * matPoints[0,:]) + (-mu_2* np.pi * np.sin(mu_2 * np.pi * matPoints[0,:]) * np.sin(mu_2 * np.pi * matPoints[1,:]))
	return values.ctypes.data
#############
def Stokes_pressure_exactSolution(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.sin(2.0 * np.pi * matPoints[0,:]) * np.cos(2.0 * np.pi * matPoints[1,:])
	return values.ctypes.data

def Stokes_speed_exactSolution_1(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = +0.5 * np.sin(2.0 * np.pi * matPoints[0,:]) * np.sin(2.0 * np.pi * matPoints[0,:]) * np.sin(2.0 * np.pi * matPoints[1,:]) * np.cos(2.0 * np.pi * matPoints[1,:])
	return values.ctypes.data

def Stokes_speed_exactSolution_2(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = -0.5 * np.sin(2.0 * np.pi * matPoints[1,:]) * np.sin(2.0 * np.pi * matPoints[1,:]) * np.sin(2.0 * np.pi * matPoints[0,:]) * np.cos(2.0 * np.pi * matPoints[0,:])
	return values.ctypes.data

### Discretization

In [ ]:
order = 1
meshSize = 0.001

In [ ]:
domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [2,3,4,5], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize }
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib)

In [ ]:
gedim.PlotMesh(mesh)

### High Fidelity approximation

In [ ]:
pressure_discreteSpace = { 'Order': 1, 'Type': 1, 'BoundaryConditionsType': [1, 2, 1, 1, 1, 1] }
speed_discreteSpace = { 'Order': 2, 'Type': 1, 'BoundaryConditionsType': [1, 2, 2, 2, 2, 2] }

[pressure_problemData, pressure_dofs, pressure_strongs] = gedim.Discretize(pressure_discreteSpace, lib)
[speed_problemData, speed_dofs, speed_strongs] = gedim.Discretize(speed_discreteSpace, lib)

In [ ]:
pressure_n_dofs = pressure_problemData['NumberDOFs']
pressure_n_strongs = pressure_problemData['NumberStrongs']
speed_n_dofs = speed_problemData['NumberDOFs']
speed_n_strongs = speed_problemData['NumberStrongs']

In [ ]:
[J_X_1, J_X_D_1] = gedim.AssembleStiffnessMatrix_Shift(speed_problemData['SpaceIndex'], speed_problemData['SpaceIndex'], Stokes_v, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_strongs + pressure_n_strongs, 0, 0, 0, lib)
[J_X_2, J_X_D_2] = gedim.AssembleStiffnessMatrix_Shift(speed_problemData['SpaceIndex'], speed_problemData['SpaceIndex'], Stokes_v, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_strongs + pressure_n_strongs, speed_n_dofs, speed_n_dofs, speed_n_strongs, lib)

[J_B_1, J_B_D_1] = gedim.AssembleAdvectionMatrix_Shift(speed_problemData['SpaceIndex'], pressure_problemData['SpaceIndex'], Stokes_advection_1, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_strongs + pressure_n_strongs, 2 * speed_n_dofs, 0, 0, lib)
[J_B_2, J_B_D_2] = gedim.AssembleAdvectionMatrix_Shift(speed_problemData['SpaceIndex'], pressure_problemData['SpaceIndex'], Stokes_advection_2, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_dofs + pressure_n_dofs, 2 * speed_n_strongs + pressure_n_strongs, 2 * speed_n_dofs, speed_n_dofs, speed_n_strongs, lib)

mu_2 = 2

J_f_1 = gedim.AssembleForcingTerm(Stokes_f_1, speed_problemData, lib)
J_f_2 = gedim.AssembleForcingTerm(Stokes_f_2, speed_problemData, lib)
J_f = np.concatenate([J_f_1, J_f_2, np.zeros(pressure_n_dofs)])

p_D = gedim.AssembleStrongSolution(Stokes_pressure_exactSolution, 1, pressure_problemData, lib)

In [ ]:
solution = gedim.LUSolver(J_X_1 + J_X_2 - J_B_1 - J_B_2 - np.transpose(J_B_1) - np.transpose(J_B_2), J_f, lib)
u = solution[0:2 * speed_n_dofs]
p = solution[2 * speed_n_dofs:]

In [ ]:
gedim.PlotSolution(mesh, pressure_dofs, pressure_strongs, p, p_D, "Pressure")
gedim.PlotSolution(mesh, speed_dofs, speed_strongs, u[0:speed_n_dofs], np.zeros(speed_n_strongs), "Speed X")
gedim.PlotSolution(mesh, speed_dofs, speed_strongs, u[speed_n_dofs:], np.zeros(speed_n_strongs), "Speed Y")
gedim.PlotSolution(mesh, speed_dofs, speed_strongs, np.sqrt(u[0:speed_n_dofs] * u[0:speed_n_dofs] + u[speed_n_dofs:] * u[speed_n_dofs:]), np.zeros(speed_n_strongs), "Speed Magnitude")

**wPOD ideas**

Let us define the parameters for the POD, i.e. the snapshot number and the parametric space. The training set is related to the parameter distribution. We use the [$\beta$-distribution](https://vitalflux.com/beta-distribution-explained-with-python-examples/).

Below you can find a way to define the training set with respect to the parameter distribution.

Play with it, use different distributions if you want.

In [ ]:
### define the training set
from scipy.stats import beta

snapshot_num = 100
mu1_range = [1., 10.]
mu2_range = [1., 3.]

P = np.array([mu1_range, mu2_range])
training_set_1 = beta.rvs(75, 75, size=(snapshot_num))
t1 = training_set_1*(mu1_range[1] - mu1_range[0]) + mu1_range[0] # beta is defined between [0,1]
training_set_2 = beta.rvs(75, 75, size=(snapshot_num))
t2 = training_set_2*(mu2_range[1] - mu2_range[0]) + mu2_range[0]
training_set = np.column_stack((t1,t2)) 


pdf1 = beta.pdf(training_set_1, 75, 75)
pdf2 = beta.pdf(training_set_2, 75, 75)

pdf_new = pdf1*pdf2 #total pdf




Here, we define the matrices needed to compute the supremizer for each solution of the Stokes problem.

In [ ]:
[X_1, XStrong_1] = gedim.AssembleStiffnessMatrix_Shift(speed_problemData['SpaceIndex'], speed_problemData['SpaceIndex'], Stokes_v, 2 * speed_n_dofs, 2 * speed_n_dofs, 2 * speed_n_strongs, 0, 0, 0, lib)
[X_2, XStrong_2] = gedim.AssembleStiffnessMatrix_Shift(speed_problemData['SpaceIndex'], speed_problemData['SpaceIndex'], Stokes_v, 2 * speed_n_dofs, 2 * speed_n_dofs, 2 * speed_n_strongs, speed_n_dofs, speed_n_dofs, speed_n_strongs, lib)

[B_1, BStrong_1] = gedim.AssembleAdvectionMatrix_Shift(speed_problemData['SpaceIndex'], pressure_problemData['SpaceIndex'], Stokes_advection_1, pressure_n_dofs, 2 * speed_n_dofs, 2 * speed_n_strongs, 0, 0, 0, lib)
[B_2, BStrong_2] = gedim.AssembleAdvectionMatrix_Shift(speed_problemData['SpaceIndex'], pressure_problemData['SpaceIndex'], Stokes_advection_2, pressure_n_dofs, 2 * speed_n_dofs, 2 * speed_n_strongs, 0, speed_n_dofs, speed_n_strongs, lib)

We now apply can apply the **weighted partitioned POD**. To do so, a classical strategy is to define a weighted snapshot of the form
$\sqrt{\rho(\boldsymbol \mu)}\boldsymbol u(\boldsymbol \mu)$ (the same holds for pressure and supremizer), where $\rho(\boldsymbol \mu) = \rho(\mu_1)\rho(\mu_2)$.

After building the weighted snapshots matrices, apply partitioned POD. Compare the results with the ones of Lab11.



In [ ]:
#### snapshot matrices creation.... YOUR TURN


  

In [ ]:
inner_product_u = X_1 + X_2

Below, we define a function that, given a covariance matrix (the maximum number of basis functions and a tolerance) computes the related eigenvalues and eigenvectors, returns the eigenvectors and the basis number.

In [ ]:
def eig_analysis(C, N_max=None, tol=1e-9):
  L_e, VM_e = np.linalg.eig(C)
  eigenvalues = []
  eigenvectors = []


  #### check

  for i in range(len(L_e)):
    eig_real = L_e[i].real
    eig_complex = L_e[i].imag
    assert np.isclose(eig_complex, 0.)
    eigenvalues.append(eig_real)
    eigenvectors.append(VM_e[i].real)


  total_energy = sum(eigenvalues)
  retained_energy_vector = np.cumsum(eigenvalues)
  relative_retained_energy = retained_energy_vector/total_energy


  if all(flag==False for flag in relative_retained_energy>= tol) and N_max != None:
    N = N_max
  else:
    N = np.argmax(relative_retained_energy >= tol) + 1
  
  return N, eigenvectors



In [ ]:
### covariance matrix
C_u = snapshot_matrix_u @ inner_product_u @ np.transpose(snapshot_matrix_u)
C_s = snapshot_matrix_s @ inner_product_u @ np.transpose(snapshot_matrix_s)
C_p = snapshot_matrix_p @ np.transpose(snapshot_matrix_p)

N_u, eigs_u = eig_analysis(C_u, N_max=N_max, tol=tol)
N_s, eigs_s = eig_analysis(C_s, N_max=N_max, tol=tol)
N_p, eigs_p = eig_analysis(C_p, N_max=N_max, tol=tol)

print(N_u, N_s, N_p)





Now we create a function that creates the basis, given the snapshots matrix, the reduced dimension and the eigenvectors.

In [ ]:
def create_basis_functions_matrix(N, snapshot_matrix, eigenvectors, inner_product=None):
  
  basis_functions = []
  
  for n in range(N):
    eigenvector =  eigenvectors[n]
    basis = np.transpose(snapshot_matrix)@eigenvector
    if inner_product!= None:
      norm = np.sqrt(np.transpose(basis) @ inner_product @ basis) ## metti inner product
    else:
      norm = np.sqrt(np.transpose(basis) @ basis)
    basis /= norm
    basis_functions.append(np.copy(basis))

  basis_function_matrix = np.transpose(np.array(basis_functions))
  
  return basis_function_matrix

We create three separate basis functions and then the global basis function that we need for the projection:

$$\mathbb {B} = 
\begin{bmatrix} \mathbb B_u \cup \mathbb B_s & 0\\
0 & \mathbb B_p
\end{bmatrix}.$$


In [ ]:
basis_functions_u = create_basis_functions_matrix(N_u, snapshot_matrix_u, eigs_u, inner_product=inner_product_u)
basis_functions_s = create_basis_functions_matrix(N_s, snapshot_matrix_s, eigs_s, inner_product=inner_product_u)
basis_functions_p = create_basis_functions_matrix(N_p, snapshot_matrix_p, eigs_p)


In [ ]:
print(basis_functions_u.shape)
print(basis_functions_p.shape)
print(basis_functions_u.shape[0] + basis_functions_p.shape[0])
print(basis_functions_p.shape)
print(solution.shape)

global_basis_function_matrix = np.zeros((basis_functions_u.shape[0] + basis_functions_p.shape[0],N_u + N_s + N_p))
global_basis_function_matrix[0:basis_functions_u.shape[0], 0:N_u] = basis_functions_u
global_basis_function_matrix[0:basis_functions_u.shape[0], N_u : N_u + N_s] = basis_functions_s
global_basis_function_matrix[basis_functions_u.shape[0]:, N_u + N_s:] = basis_functions_p
print(global_basis_function_matrix.shape)

global_basis_function_matrix_no_sipremizer = np.zeros((basis_functions_u.shape[0] + basis_functions_p.shape[0],N_u + N_p))
global_basis_function_matrix_no_sipremizer[0:basis_functions_u.shape[0], 0:N_u] = basis_functions_u
global_basis_function_matrix_no_sipremizer[basis_functions_u.shape[0]:, N_u:] = basis_functions_p

We now define the assemble-functions

In [ ]:
def assemble_reduced_matrix(basis, fom_matrix):
  return np.transpose(basis) @ (fom_matrix) @ basis

def assemble_reduced_vector(basis, fom_vector):
  return np.transpose(basis) @ (fom_vector)

Let us finish the offline phase

In [ ]:
### ASSEMBLE REDUCED SYSTEMS
reduced_stiff_Stokes = assemble_reduced_matrix(global_basis_function_matrix, (J_X_1 + J_X_2)) 
reduced_divergence_operator_1 = assemble_reduced_matrix(global_basis_function_matrix, (J_B_1)) 
reduced_divergence_operator_2 = assemble_reduced_matrix(global_basis_function_matrix, (J_B_2))



We are ready for a new evaluation!

In [ ]:
### New eval
thetaA1 = 1
mu_2 = 2
J_f_1 = gedim.AssembleForcingTerm(Stokes_f_1, speed_problemData, lib)
J_f_2 = gedim.AssembleForcingTerm(Stokes_f_2, speed_problemData, lib)
J_f = np.concatenate([J_f_1, J_f_2, np.zeros(pressure_n_dofs)])
reduced_lhs = thetaA1*reduced_stiff_Stokes - reduced_divergence_operator_1 - reduced_divergence_operator_2 - np.transpose(reduced_divergence_operator_1) - np.transpose(reduced_divergence_operator_2)
reduced_rhs = assemble_reduced_vector(global_basis_function_matrix, J_f)

In [ ]:
reduced_solution = np.linalg.solve(reduced_lhs, reduced_rhs)
print(reduced_solution)

In [ ]:
###### plot #######
reduced_u_dof = N_u + N_s
# reduced_p_dof = N_p
reduced_solution_FE_basis = global_basis_function_matrix @ reduced_solution
reduced_u = reduced_solution_FE_basis[0:2*speed_n_dofs]
reduced_p = reduced_solution_FE_basis[2*speed_n_dofs:]

gedim.PlotSolution(mesh, pressure_dofs, pressure_strongs, reduced_p, p_D, "Pressure")
gedim.PlotSolution(mesh, speed_dofs, speed_strongs, np.sqrt(reduced_u[0:speed_n_dofs] * reduced_u[0:speed_n_dofs] + reduced_u[speed_n_dofs:] * reduced_u[speed_n_dofs:]), np.zeros(speed_n_strongs), "Speed Magnitude")


**What happens without supremizer?**

In [ ]:
### NO SUPREMIZER
reduced_stiff_Stokes_nsup = assemble_reduced_matrix(global_basis_function_matrix_no_sipremizer, (J_X_1 + J_X_2)) # np.transpose(global_basis_function_matrix) @ (J_X_1 + J_X_2) @ global_basis_function_matrix
reduced_divergence_operator_1_nsup = assemble_reduced_matrix(global_basis_function_matrix_no_sipremizer, (J_B_1)) #np.transpose(global_basis_function_matrix) @ J_B_1 @ global_basis_function_matrix
reduced_divergence_operator_2_nsup = assemble_reduced_matrix(global_basis_function_matrix_no_sipremizer, (J_B_2))
# - J_B_1 - J_B_2


In [ ]:
### New eval
reduced_lhs = thetaA1*reduced_stiff_Stokes_nsup - reduced_divergence_operator_1_nsup - reduced_divergence_operator_2_nsup - np.transpose(reduced_divergence_operator_1_nsup) - np.transpose(reduced_divergence_operator_2_nsup)
reduced_rhs = assemble_reduced_vector(global_basis_function_matrix_no_sipremizer, J_f)

In [ ]:
reduced_solution = np.linalg.solve(reduced_lhs, reduced_rhs)
print(reduced_solution)

In [ ]:
###### plot #######
reduced_u_dof = N_u + N_s
# reduced_p_dof = N_p
reduced_solution_FE_basis = global_basis_function_matrix_no_sipremizer @ reduced_solution
reduced_u = reduced_solution_FE_basis[0:2*speed_n_dofs]
reduced_p = reduced_solution_FE_basis[2*speed_n_dofs:]

gedim.PlotSolution(mesh, pressure_dofs, pressure_strongs, reduced_p, p_D, "Pressure")
gedim.PlotSolution(mesh, speed_dofs, speed_strongs, np.sqrt(reduced_u[0:speed_n_dofs] * reduced_u[0:speed_n_dofs] + reduced_u[speed_n_dofs:] * reduced_u[speed_n_dofs:]), np.zeros(speed_n_strongs), "Speed Magnitude")


Let us analyze the usual stuff: errors and speedups!
Below you find a function that computes the error.

In [ ]:
######### def error functions ######

def compute_error(fom_solution, rom_solution_FE_basis, inner_product=None, type_err="relative"):
    
    error_function_u = fom_solution - rom_solution_FE_basis
    
    if inner_product == None:
        inner_product_matrix = np.identity(fom_solution.shape[0])
    else:
      print()
      inner_product_matrix = inner_product
    
    error_norm_squared_component = np.transpose(error_function_u) @ inner_product_matrix @ error_function_u
    absolute_error = np.sqrt(abs(error_norm_squared_component))
    
    if type_err == "absolute":
      
      return absolute_error
    
    else:
      full_solution_norm_squared_component = np.transpose(fom_solution) @  inner_product_matrix @ fom_solution
      relative_error = absolute_error/np.sqrt(abs(full_solution_norm_squared_component))
    
      return relative_error
    

In [ ]:
### compute error... YOUR TURN


In [ ]:
print("avarege relative error for velocity = ", np.mean(rel_err_u) )
print("avarege absolute error for velocity = ", np.mean(abs_err_u) )

print("avarege relative error for pressure = ", np.mean(rel_err_p) )
print("avarege absolute error for pressure = ", np.mean(abs_err_p) )


print("avarege speed_up = ", np.mean(speed_up) )